<p style="padding: 10px; border: 1px solid black;">
<img src="images/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>


# <a name="0">MLU Advanced Prompt Engineering for LLMs</a>
## <a name="0">Lab 6: Guardrails</a>

This notebook demonstrates how to use various techniques that can help improve the safety and security of LLM-backed applications. The coding examples cover guardrails that can filter certain keywords or that leverage metrics to decide if content is harmful.

1. <a href="#1">Install and import libraries</a>
2. <a href="#2">Set up Bedrock for inference</a>
3. <a href="#3">Guardrails</a>
    - <a href="#31">Keyword-based filtering</a>
    - <a href="#32">Metric-based filtering</a>
    - <a href="#33">LLM-based filtering</a>
4. <a href="#4">Conclusion</a>
    
Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.

---

<br/>
You will be presented with coding activities to check your knowledge and understanding throughout the notebook whenever you see the MLU robot:

<img style="display: block; margin-left: auto; margin-right: auto;" src="./images/activity.png" alt="Activity" width="125"/>


## <a name="1">1. Install and import libraries</a>
(<a href="#0">Go to top</a>)

Let's start by installing all required packages as specified in the `requirements.txt` file and importing several libraries.

In [1]:
%pip install -q --upgrade pip
!pip3 install -r requirements.txt --quiet

In [2]:
import warnings, sys

warnings.filterwarnings("ignore")

import json
from IPython.display import Markdown

## <a name="2">2. Set up Bedrock for inference</a>
(<a href="#0">Go to top</a>)

To get started, set up Bedrock and instantiate an active runtime to query LLMs.

In [3]:
import boto3

# define the bedrock-runtime client that will be used for inference
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

# define the model
bedrock_model_id = "anthropic.claude-v1"

# each model has a different set of inference parameters
inference_modifier = {
    "max_tokens_to_sample": 300,
    "temperature": 0,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"],
}

In [4]:
from langchain.llms.bedrock import Bedrock

# define the langchain module with the selected bedrock model
bedrock_llm = Bedrock(
    model_id=bedrock_model_id,
    client=bedrock_runtime,
    model_kwargs=inference_modifier,
)

Next, use Bedrock for inference to test everything works as expected:

In [5]:
bedrock_llm("\n\nHuman: How are you doing today? \n\nAssistant:")

" I'm doing well, thanks for asking! My name is Claude."

## <a name="3">3. Guardrails</a>
(<a href="#0">Go to top</a>)


### Dealing with prompt injections
What can be done to prevent prompt injections? In this section, you will be working with guardrails.

Whenever guardrails flag a potentially harmful output, there are several strategies to deal with this and to improve security and safety of LLMs:
- refuse to perform task (prompt refusal)
- perform task, but add disclaimer
- summarize result in a harmless way
- explain and perform a similar but harmless task

Let's have a look at [Guardrails.ai](https://docs.guardrailsai.com/) in combination with [LangChain](https://www.langchain.com/):

#### Guardrails
To get started with [Guardrails.ai](https://docs.guardrailsai.com/) you need a RAIL (Reliable AI markup Language) spec XML:

```
<rail version="0.1">

    <output> 
    ...
    </output>


    <prompt> 
    ...
    </prompt>

</rail>
```

The main components are:
- `output` contains information about the expected output of the LLM such as overall structure of the LLM output, type info for each field, and the quality criteria for each field and the corrective action to be taken in case quality criteria is not met
- `prompt` contains the high level instructions that are sent to the LLM


In [6]:
rail_spec_sample = """
<rail version="0.1">
    <output>
        <object name="product_info">
            <string description="Brand name of product" name="brand"></string>
            <float description="Price of product" name="price" format="valid-range: 0 1000"></float>
            <string description="Description of product" name="description"></string>
        </object>
    </output>

    <prompt>
    \n\nHuman: Given the following text snippet, extract a dictionary that contains the product's information.
    ${product_notes}
    ${gr.complete_json_suffix}
    \n\nAssistant:
    </prompt>
</rail>
"""

#### LangChain

Let's include the RAIL spec in the code to add a layer of security around LangChain components.

In [7]:
import guardrails as gd

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The `GuardrailsOutputParser` contains a `Guard` object, which can be used to access the prompt and output schema.

In [8]:
from langchain.output_parsers import GuardrailsOutputParser

output_parser_simple = GuardrailsOutputParser.from_rail_string(
    rail_spec_sample, api=bedrock_llm
)

You can now create a [LangChain](https://www.langchain.com/) `PromptTemplate` from this output parser.

In [9]:
from langchain.prompts import PromptTemplate

prompt_simple = PromptTemplate(
    template=output_parser_simple.guard.prompt.escape(),
    input_variables=output_parser_simple.guard.prompt.variable_names,
)

Finally, pass the prompt to Bedrock and retrieve the output.

In [10]:
extract_info = """19.99. A cold-weather Carhartt classic hat that is made of stretchy rib knit that's soft to the touch."""

output_simple = bedrock_llm(
    prompt_simple.format_prompt(product_notes=extract_info).to_string()
)

In [11]:
from rich import print

print(output_parser_simple.parse(output_simple))

{
    'product_info': {
        'brand': 'Carhartt',
        'price': 19.99,
        'description': "A cold-weather Carhartt classic hat that is made of stretchy rib knit that's soft to the 
touch."
    }
}


<div class="alert alert-block alert-warning">
<b>Try different prompts to extract product information.</b> For example, you could try this snippet of text and parse it: </br> <code>"Premier Protein Shake with 24 Vitamins Minerals Nutrients to Support Immune Health. Price:$28.48 ($0.21 / Fl Oz)"</code>
</div>
<img style="display: block; margin-left: auto; margin-right: auto;" src="./images/activity.png" alt="Activity" width="125"/>


In [12]:
############## CODE HERE ####################


############## END OF CODE ##################

### <a name="31">3.1. Keyword-based input filtering</a>
(<a href="#3">Go to "Guardrails"</a>)

Keyword-based filtering checks a users' input for forbidden words or phrases. If keywords are identified, the prompt will be rejected. However, this can be bypassed by using alternative words and might be difficult to implement across different tasks, domains etc.

[Guardrails.ai](https://docs.guardrailsai.com/) offers different strategies on how to deal with failed validation checks; e.g. filtering, fixing, asking the LLM again. A full overview of all the possible corrective actions can be found [here](https://docs.guardrailsai.com/concepts/output/#specifying-corrective-actions). In the code snippet below, you will see a custom validator that checks for keywords in the LLM output and replaces them with `***`.

In [13]:
from guardrails.validators import (
    Validator,
    register_validator,
    ValidationResult,
    PassResult,
    FailResult,
)
from typing import Dict, Any


@register_validator(name="is-keyword-free", data_type="string")
class IsKeywordFree(Validator):
    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        kw_list = ["hate"]
        # check for forbidden words
        if any(kw in value for kw in kw_list):
            # replace forbidden words in output with ***
            for kw in kw_list:
                censored_text = value.replace(kw, "***")
            # return result
            return FailResult(
                error_message=f"Expression '{value}' contains forbidden keyword.",
                fix_value=censored_text,
            )
        # else return pass
        return PassResult()

Let's translate a statement this time; inside the XML you need to specify `format` and the action to take `on-fail`. For `format` you pass in the custom validator that you just instantiated; for the corrective action, try `fix`.

In [14]:
rail_spec_filter = """
<rail version="0.1">
    <output>
        <string description="Translate to English" format="is-keyword-free" name="translated_statement" on-fail-is-keyword-free="fix"></string>
    </output>

    <prompt>
    \n\nHuman: Translate the given statement into English language:
    ${statement_to_be_translated}
    ${gr.complete_json_suffix}
    \n\nAssistant:
    </prompt>

</rail>
"""

Next, you need the `Guard` object; you can create this directly from the RAIL spec or use LangChain again.

In [15]:
# create a Guard object directly from a RAIL string
guard_keyword = gd.Guard.from_rail_string(rail_spec_filter)

# provide API to Guard and the prompt input
raw_llm_response, validated_response = guard_keyword(
    llm_api=bedrock_llm,
    prompt_params={"statement_to_be_translated": "Ich hasse dich."},
)

# show the output
print(f"Validated Output: {validated_response}")

Validated Output: {'translated_statement': 'I *** you.'}

Alternatively, you can use LangChain again.

In [16]:
# instantiate the output parser
output_parser_filter = GuardrailsOutputParser.from_rail_string(
    rail_spec_filter, api=bedrock_llm
)

# initiate the prompt template
prompt_template_filter = PromptTemplate(
    template=output_parser_filter.guard.prompt.escape(),
    input_variables=output_parser_filter.guard.prompt.variable_names,
)

# define a sentence to translate
translate_this = "Ich hasse dich."

# apply the prompt template and call the model
output_filter = bedrock_llm(
    prompt_template_filter.format_prompt(
        statement_to_be_translated=translate_this
    ).to_string()
)

# print the validated result
print(output_parser_filter.parse(output_filter))

{'translated_statement': 'I *** you.'}

<div class="alert alert-block alert-warning">
<b>Write your own custom validator to check that the response length does not exceed a limit.</b> If the limit is exceeded, the LLM should be reasked to create a new and shorter response. You can achieve this by using <code>reask</code> for the <code>on-fail</code> condition.
</div>
<img style="display: block; margin-left: auto; margin-right: auto;" src="./images/activity.png" alt="Activity" width="125"/>

In [17]:
############## CODE HERE ####################


############## END OF CODE ##################

### <a name="32">3.2. Metric-based input filtering</a>
(<a href="#3">Go to "Guardrails"</a>)

Metric-based filtering uses evaluation metrics like perplexity, toxicity, or polarity to detect unnatural or harmful inputs. A threshold determines if the prompt will be executed. However, metrics are not infallible and there could be significant impact on latency.

In [18]:
from profanity_check import predict


@register_validator(name="is-profanity-free", data_type="string")
class IsProfanityFree(Validator):
    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        prediction = predict([value])
        if prediction[0] == 1:
            return FailResult(
                error_message=f"The result contains profanity and will be filtered.",
                fix_value="",
            )
        return PassResult()

In [19]:
rail_spec_metric = """
<rail version="0.1">
    <output>
        <string description="Translate to English" format="is-profanity-free" name="translated_statement" on-fail-is-profanity-free="filter"></string>
    </output>

    <prompt>
    \n\nHuman: Translate the given statement into English language:
    ${statement_to_be_translated}
    ${gr.complete_json_suffix}
    \n\nAssistant:
    </prompt>

</rail>
"""

In [20]:
guard_metric = gd.Guard.from_rail_string(rail_spec_metric)

raw_llm_response, validated_response = guard_metric(
    llm_api=bedrock_llm,
    prompt_params={"statement_to_be_translated": "Ich hasse dich."},
)

print(f"Validated Output: {validated_response}")

Validated Output: {}

In [21]:
print(guard_metric.state.most_recent_call.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Human: Translate the given statement into English language:                                             │ │
    │ │     Ich hasse dich.                                                                                     │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string description="Translate to English" format="is-profanity-free" name="translated_statement"/> │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere,     │ │
    │ │ enter `null`.                                                                                           │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`                        │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Assistant:                                                                                              │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭────────────────────────────────────────────

There are certain metrics that are so common, that pre-built validators were created.

<div class="alert alert-block alert-warning">
<b>Write a custom metric-based guardrail or use some of the pre-built <a href=https://docs.guardrailsai.com/api_reference/validators/>Guardrail.ai validators</a>.</b>
You can load them in with <code>from guardrails.validators import IsProfanityFree, RemoveRedundantSentences, ValidLength</code>.
</div>
<img style="display: block; margin-left: auto; margin-right: auto;" src="./images/activity.png" alt="Activity" width="125"/>

In [22]:
############## CODE HERE ####################


############## END OF CODE ##################

### <a name="33">3.3. LLM-based and model-based filtering</a>
(<a href="#3">Go to "Guardrails"</a>)

LLM-based filtering utilizes another LLM to determine the intent of the request; if the helper LLM finds that the prompt is malicious, the prompt will be rejected and not sent to the real LLM. To set this up, the output of one LLM needs to be provided as input to the next LLM - this is called **prompt chaining**. Careful, as doing this in a production scenario with thousands of requests might incur significant cost, latency issues and cannot guarantee that harm will be prevented. 

#### Prompt chaining using LLMs
Prompt chaining is a method of using LLMs to accomplish a task by breaking it into multiple smaller prompts and passing the output of one prompt as the input to the next. Chaining multiple prompts can help achieve a desired format and quality in the generated text. The very first model in the chain should be an intent classifier, such as [Cohere's Intent Recognition](https://docs.cohere.com/reference/intent-recognition).

<div class="alert alert-block alert-warning">
<b>Write a simple class or method that passes a prompt through a classifier with an added decision criteria that determines whether or not the prompt should be returned to user, rephrased or filtered.</b>
</div>
<img style="display: block; margin-left: auto; margin-right: auto;" src="./images/activity.png" alt="Activity" width="125"/>

In [23]:
############## CODE HERE ####################


############## END OF CODE ##################

#### Preventing prompt leakage example
Prompt leaking is another type of prompt injection where prompt attacks are designed to leak details from the prompt which could contain confidential or proprietary information that was not intended for the public. Using Amazon Comprehend, you will add a `AmazonComprehendModerationChain` moderation which requires some configuration to configuring the behavior of the PII validations: `ModerationPiiConfig`.

In [24]:
from langchain_experimental.comprehend_moderation import (
    AmazonComprehendModerationChain,
    ModerationPiiConfig,
)

comprehend_client = boto3.client("comprehend", region_name="us-east-1")

pii_config = ModerationPiiConfig(labels=["SSN"], redact=True, mask_character="X")

comprehend_moderation = AmazonComprehendModerationChain(
    moderation_config=pii_config, client=comprehend_client, verbose=True
)


In addition to the moderation chain, you  need a fake LLM class that can be used for testing. This allows you to mock out calls to the LLM and simulate what would happen if the LLM responded in a certain way. LangChain provides this as `FakeListLLM`.


In [25]:
from langchain.prompts import PromptTemplate
from langchain.llms.fake import FakeListLLM
from langchain_experimental.comprehend_moderation.base_moderation_exceptions import (
    ModerationPiiError,
)

template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

responses = [
    "Final Answer: A credit card number looks like 1289-2321-1123-2387. A fake SSN number looks like 323-22-9980. John Doe's phone number is (999)253-9876.",
    # replace with your own expletive
    "Final Answer: This is a really <expletive> way of constructing a birdhouse. This is <expletive> insane to think that any birds would actually create their <expletive> nests here.",
]

llm = FakeListLLM(responses=responses)

pii_chain = (
    prompt
    | comprehend_moderation
    | {"input": (lambda x: x["output"]) | llm}
    | comprehend_moderation
)

try:
    response = pii_chain.invoke(
        {
            "question": "A sample SSN number looks like this 123-45-7890. Can you give me some more samples?"
        }
    )
except Exception as e:
    print(str(e))
else:
    print(response["output"])



> Entering new AmazonComprehendModerationChain chain...
Running AmazonComprehendModerationChain...
Running pii Validation...
Found PII content..stopping..
The prompt contains PII entities and cannot be processed


The prompt contains PII entities and cannot be processed

## <a name="4">4. Conclusion</a>

- Use clear and specific instructions. The prompt should be clear and specific about what you are asking the LLM to do. This will help to avoid ambiguity and ensure that the LLM generates a relevant and unbiased response.
- Segregate external content from user prompts. Separate and denote where untrusted content is being used to limit their influence on user prompts. 
- Use a variety of prompts. Don’t just use the same prompt every time. This will help to ensure that the LLM is exposed to a variety of viewpoints and doesn’t become biased towards any one particular perspective.

### Additional resources
- https://github.com/microsoft/promptbench
- https://www.promptingguide.ai/techniques
- https://github.com/uptrain-ai/uptrain

# Thank you!

<p style="padding: 10px; border: 1px solid black;">
<img src="images/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>